# Carregamento da base de dados

In [2]:
#Importando Bibiliotecas
import pandas as pd
print('Importando Bibliotecas...')
import time
import psycopg2
import mysql.connector
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import urllib
print('Bibliotecas importadas')

Importando Bibliotecas...
Bibliotecas importadas


In [5]:
#Download dos dados da ANATEL
import wget
url='https://www.anatel.gov.br/dadosabertos/paineis_de_dados/meu_municipio/meu_municipio.zip'
wget.download(url)

100% [..........................................................................] 9932575 / 9932575

'meu_municipio.zip'

# Pré-Analise dos dados

In [6]:
#Descompactação dos arquivos baixados em uma pasta chamada outputanatelacesos.
import os
import re
from zipfile import ZipFile
if os.path.isdir("outputanatelacesos"):
    print("O diretório output já existe!")
else:
    print("O diretório output não existe!, criando diretório")
    nome_arquivo = "meu_municipio.zip"

    with ZipFile(nome_arquivo, 'r') as zip: 
        zip.extractall('outputanatelacesos') 
        print('Feito!') 


O diretório output não existe!, criando diretório
Feito!


In [7]:
#Abrindo planilha por planilha, extraindo e amarzenando apenas dados de acessos do Pará no dataframe

temp = pd.read_csv("outputanatelacesos/Meu_Municipio_Acessos.csv",encoding="UTF-8", sep=';', low_memory=False)
dataframe = pd.DataFrame(temp)
servicos_acessos = dataframe[dataframe['UF'] == 'PA' ]

servicos_acessos.head(300)


,Ano,Mês,Acessos,Serviço,Densidade,Código IBGE,Município,UF,Nome UF,Região,Código Nacional
12,2022,12,41163,Telefonia Móvel,"82,381793619453",1502707,Conceição do Araguaia - PA,PA,Pará,Norte,94
13,2022,12,6412,Banda Larga Fixa,"13,326405486854",1502707,Conceição do Araguaia - PA,PA,Pará,Norte,94
14,2022,12,2339,TV por Assinatura,"4,8612698742596",1502707,Conceição do Araguaia - PA,PA,Pará,Norte,94
15,2022,12,235,Telefonia Fixa,"0,488413176764",1502707,Conceição do Araguaia - PA,PA,Pará,Norte,94
188,2022,12,10231,Telefonia Móvel,"57,343224759476",1501576,Bom Jesus do Tocantins - PA,PA,Pará,Norte,94
...,...,...,...,...,...,...,...,...,...,...,...
8706,2022,12,44,Telefonia Fixa,"0,13946559320422",1508035,Tracuateua - PA,PA,Pará,Norte,91
8715,2022,12,23213,Telefonia Móvel,"45,406230910202",1505486,Pacajá - PA,PA,Pará,Norte,91
8716,2022,12,2529,Banda Larga Fixa,"5,149664019548",1505486,Pacajá - PA,PA,Pará,Norte,91
8717,2022,12,1113,TV por Assinatura,"2,2663408674404",1505486,Pacajá - PA,PA,Pará,Norte,91


In [8]:
#Abrindo planilha por planilha, extraindo e amarzenando apenas dados de cobertura de rede móvel do Pará no dataframe

temp1 = pd.read_csv("outputanatelacesos/Meu_Municipio_Cobertura.csv",encoding="UTF-8", sep=';', low_memory=False)
dataframe1 = pd.DataFrame(temp1)
cobertura_rede_movel = dataframe1[dataframe1['UF'] == 'PA' ]

cobertura_rede_movel.head(300)

,Ano,Mês,Operadora,Tecnologia,Código IBGE,Município,UF,Nome UF,Região,Código Nacional,% moradores cobertos,% domicílios cobertos,% área coberta
189,2022.0,12.0,OI,5G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,0,0,0
190,2022.0,12.0,SERCOMTEL,3G4G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,0,0,0
191,2022.0,12.0,VIVO,3G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,"19,778294936235","19,473806589901","3,6097347920042"
192,2022.0,12.0,CLARO,4G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,"56,110364571265","54,93242103789","3,2702347436482"
193,2022.0,12.0,Todas,3G4G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8611,2022.0,12.0,ALGAR,Todas,1505502.0,Paragominas - PA,PA,Pará,Norte,91.0,0,0,0
8612,2022.0,12.0,ALGAR,3G4G,1505502.0,Paragominas - PA,PA,Pará,Norte,91.0,0,0,0
8613,2022.0,12.0,ALGAR,3G,1505502.0,Paragominas - PA,PA,Pará,Norte,91.0,0,0,0
8614,2022.0,12.0,Todas,2G,1505502.0,Paragominas - PA,PA,Pará,Norte,91.0,"60,242702292325","61,374603532663","5,5489401052107"


# Ajuste na tipagem dos dados

In [21]:
#deixando as colunas em caixa baixa e substituindo espaços dos dados servicos_acessos
from unidecode import unidecode
teste = list(servicos_acessos.columns)
colunas =[]
for x in teste:
    semmaiuscula = unidecode(x.lower())
    repl = semmaiuscula.replace(" ", "_")
    print(repl)
    colunas.append(repl)
    
print(colunas)

servicos_acessos.columns = colunas
servicos_acessos.head(300)

ano
mes
acessos
servico
densidade
codigo_ibge
municipio
uf
nome_uf
regiao
codigo_nacional
['ano', 'mes', 'acessos', 'servico', 'densidade', 'codigo_ibge', 'municipio', 'uf', 'nome_uf', 'regiao', 'codigo_nacional']


,ano,mes,acessos,servico,densidade,codigo_ibge,municipio,uf,nome_uf,regiao,codigo_nacional
12,2022,12,41163,Telefonia Móvel,82.381794,1502707,Conceição do Araguaia - PA,PA,Pará,Norte,94
13,2022,12,6412,Banda Larga Fixa,13.326405,1502707,Conceição do Araguaia - PA,PA,Pará,Norte,94
14,2022,12,2339,TV por Assinatura,4.861270,1502707,Conceição do Araguaia - PA,PA,Pará,Norte,94
15,2022,12,235,Telefonia Fixa,0.488413,1502707,Conceição do Araguaia - PA,PA,Pará,Norte,94
188,2022,12,10231,Telefonia Móvel,57.343225,1501576,Bom Jesus do Tocantins - PA,PA,Pará,Norte,94
...,...,...,...,...,...,...,...,...,...,...,...
8706,2022,12,44,Telefonia Fixa,0.139466,1508035,Tracuateua - PA,PA,Pará,Norte,91
8715,2022,12,23213,Telefonia Móvel,45.406231,1505486,Pacajá - PA,PA,Pará,Norte,91
8716,2022,12,2529,Banda Larga Fixa,5.149664,1505486,Pacajá - PA,PA,Pará,Norte,91
8717,2022,12,1113,TV por Assinatura,2.266341,1505486,Pacajá - PA,PA,Pará,Norte,91


In [22]:
#deixando as colunas em caixa baixa e substituindo espaços  dos dados cobertura_rede_movel
from unidecode import unidecode
teste1 = list(cobertura_rede_movel.columns)
colunas1 =[]
for x in teste1:
    semmaiuscula1 = unidecode(x.lower())
    repl1 = semmaiuscula1.replace(" ", "_")
    print(repl1)
    colunas1.append(repl1)
    
print(colunas1)

cobertura_rede_movel.columns = colunas1
cobertura_rede_movel.head(300)

ano
mes
operadora
tecnologia
codigo_ibge
municipio
uf
nome_uf
regiao
codigo_nacional
%_moradores_cobertos
%_domicilios_cobertos
%_area_coberta
['ano', 'mes', 'operadora', 'tecnologia', 'codigo_ibge', 'municipio', 'uf', 'nome_uf', 'regiao', 'codigo_nacional', '%_moradores_cobertos', '%_domicilios_cobertos', '%_area_coberta']


,ano,mes,operadora,tecnologia,codigo_ibge,municipio,uf,nome_uf,regiao,codigo_nacional,%_moradores_cobertos,%_domicilios_cobertos,%_area_coberta
189,2022,12.0,OI,5G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,0.000000,0.000000,0.000000
190,2022,12.0,SERCOMTEL,3G4G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,0.000000,0.000000,0.000000
191,2022,12.0,VIVO,3G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,19.778295,19.473807,3.609735
192,2022,12.0,CLARO,4G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,56.110365,54.932421,3.270235
193,2022,12.0,Todas,3G4G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8611,2022,12.0,ALGAR,Todas,1505502.0,Paragominas - PA,PA,Pará,Norte,91.0,0.000000,0.000000,0.000000
8612,2022,12.0,ALGAR,3G4G,1505502.0,Paragominas - PA,PA,Pará,Norte,91.0,0.000000,0.000000,0.000000
8613,2022,12.0,ALGAR,3G,1505502.0,Paragominas - PA,PA,Pará,Norte,91.0,0.000000,0.000000,0.000000
8614,2022,12.0,Todas,2G,1505502.0,Paragominas - PA,PA,Pará,Norte,91.0,60.242702,61.374604,5.548940


In [15]:
#ajustando os dados de serviços e rede movel (substituindo None por 0 em todas as tabelas e , por ., além de ajustar a tipagem de algumas colunas)

servicos_acessos.replace({None: 0}, regex=True, inplace=True)
servicos_acessos.replace({',': '.'}, regex=True, inplace=True)
cobertura_rede_movel.replace({None: 0}, regex=True, inplace=True)
cobertura_rede_movel.replace({',': '.'}, regex=True, inplace=True)

data_types_dict = {'densidade': float}
data_types_dict1 = {'%_moradores_cobertos': float}
data_types_dict2 = {'%_domicilios_cobertos': float}
data_types_dict3 = {'%_area_coberta': float}
data_types_dict4 = {'ano': int}


servicos_acessos = servicos_acessos.astype(data_types_dict)
cobertura_rede_movel = cobertura_rede_movel.astype(data_types_dict1)
cobertura_rede_movel = cobertura_rede_movel.astype(data_types_dict2)
cobertura_rede_movel = cobertura_rede_movel.astype(data_types_dict3)
cobertura_rede_movel = cobertura_rede_movel.astype(data_types_dict4)


servicos_acessos.dtypes
cobertura_rede_movel.dtypes

ano                        int32
mes                      float64
operadora                 object
tecnologia                object
codigo_ibge              float64
municipio                 object
uf                        object
nome_uf                   object
regiao                    object
codigo_nacional          float64
%_moradores_cobertos     float64
%_domicilios_cobertos    float64
%_area_coberta           float64
dtype: object

In [8]:
#Divisão dos dados de porcentagem por 100 nos dados de cobertura_rede_movel, para melhor exibição no gráfico

cobertura_rede_movel['%_moradores_cobertos'] = rslt_df1['%_moradores_cobertos'] / 100
cobertura_rede_movel['%_domicilios_cobertos'] = rslt_df1['%_domicilios_cobertos'] / 100
cobertura_rede_movel['%_area_coberta'] = rslt_df1['%_area_coberta'] / 100
cobertura_rede_movel.head()

,ano,mes,operadora,tecnologia,codigo_ibge,municipio,uf,nome_uf,regiao,codigo_nacional,%_moradores_cobertos,%_domicilios_cobertos,%_area_coberta
189,2022,12.0,OI,5G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,0.000000,0.000000,0.000000
190,2022,12.0,SERCOMTEL,3G4G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,0.000000,0.000000,0.000000
191,2022,12.0,VIVO,3G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,0.197783,0.194738,0.036097
192,2022,12.0,CLARO,4G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,0.561104,0.549324,0.032702
193,2022,12.0,Todas,3G4G,1502707.0,Conceição do Araguaia - PA,PA,Pará,Norte,94.0,0.000000,0.000000,0.000000
